In [ ]:
# Importer ipysheet, dette bruker vi for å vise dataframe som ett spreadsheet
import ipysheet

# Standardimporter
import dapla as dp
import pandas as pd

In [ ]:
# Eksempeldataframe
diam = dp.read_pandas("/felles/kurs/helt_python/diamonds")
# Til dette eksempelet bruker vi de første ti radene
diam = diam.head(10)
# Det er viktig at du begrenser dette noe, det er ikke å anbefale å sitte å mikroeditere flere hundre rader om gangen.
# Hverken for din egen del, eller for jupyters prosessering.

In [ ]:
# Log path (du må ha skriverettigheter)
microlog_path = '/felles/trash/diamonds_mikroeditlog'
# Dette vil da etter hvert bli en "samling" av log-entries. Inkludert bruker og tidspunkt.

In [ ]:
# Lag ett interaktivt regneark med ipysheets
sheet = ipysheet.from_dataframe(diam)
# Display regnearket 
sheet
# Under kan du da redigere dataen direkte i cellene

In [ ]:
# Kjør denne cellen, og videre nedover, etter at du er ferdig med å redigere.
diam_edit = ipysheet.to_dataframe(sheet)

In [ ]:
# Denne er inkorporert i funksjonen lengre ned, men om du ikke har tenkt å bruke den så må du bruke denne linjen.
#diam_edit.index = diam_edit.index.astype(diam.index.dtype)
# Det er fordi ipysheet av en eller annen grunn konverterer rad-navnene til "object", mens de ser ut til å komme fra dapla som "int".

In [ ]:
# Redigeringene bør logges, slik at vi vet hva som ble redigert i denne runden
#microlog_now = diam.compare(diam_edit)
# Dette vil gi deg det som bør logges, dvs. forskjellen mellom utgangspunktet og den redigerte dataframen.
# Under ligger en funksjon, som eksempel på hvordan en slik logg kan tas vare på

In [ ]:
# Display loggen generert over
#microlog_now

# Her vil den du starter comparen på være "self", diam.compare()
# Mens "other" vil være den i parantesen fra compare, .compare(diam_other)

In [ ]:
# Denne funksjonen tar to dataframes, men vi forventer at den første er utgangspunktet, og den andre er endret etter mikroeditering
# Path er da pathen hvor alle føringene av mikrologgen skal slås sammen.
def microlog_write(df1, df2, path):
    # Siden denne funksjonen i teorien kan kopieres inn dit man ønsker, så forsikrer vi oss bare om at vi har de importene vi trenger.
    from datetime import datetime 
    import os
    from IPython.core.display import display
    import dapla as dp
    
    # Sett index datatype på df2 til å være samme som df1, ipysheet konverterer fra int til object av en eller annen grunn.
    df2.index = df2.index.astype(df1.index.dtype)
    # Legg gjerne til flere kompensasjoner her, om du oppdager noe comparen lengre ned ikke takler med dine datasett
    
    # Variablene vi skal skrive til loggen
    # Nå-tidspunkt
    now = f'{datetime.now():%Y-%m-%d -%H:-%M}'
    # Brukernavn i jupyter
    user = os.getenv('JUPYTERHUB_USER')
    # Her gjør vi selve comparen som finner forskjellen mellom dataframene.
    microlog_now = df1.compare(df2)
    
    
    # Det som skal legges på hovedloggen.
    # Vi encoder her en hel dataframe som json, og setter den i en celle i en ny dataframe.
    microlog_add = pd.DataFrame([[now, user, microlog_now.to_json()]])
    # Dvs. om endringene (compare-dataframe) er massive nok, så kan vi få ett issue her at stringen blir for lang...
    # Navngi kolonnene, siden de nå er tomme i utgangpunktet (kanskje unødvendig, men why not)
    microlog_add.columns = ['Date', 'User', 'Json-dataframe']
    
    
    # Ikke sikkert datasettet finnes...
    try:
        # Prøv å les inn hovedlagring
        microlog_global = dp.read_pandas(path)
        # Kombiner det som ligger der, med den nye entryen
        microlog_global = pd.concat([microlog_global, microlog_add])
        # Skriv den kombinerte totale loggen tilbake til daplalagringen
        # Verdivurderingsscorenivå:    .option("valuation", "XXXX") = SENSITVE, SHIELDED, INTERNAL eller OPEN
        # Dataferdigstillingssnivå:    .option("state", "XXXX") = RAW, INPUT, PROCESSED, OUTPUT, PRODUCT, TEMP eller OTHER
        dp.write_pandas(microlog_global, path, valuation='INTERNAL', state= 'PROCESSED')
        # Print ut dataframen som skrives til loggen nå
        display(microlog_now)
    # Om den totale loggen ikke er initialisert, så si fra om det.
    except Exception as e:
        print(f'''Kanskje det ikke finnes noe datasett på denne plasseringen enda?
Dobbeltsjekk, så skriv ett tomt datasett til plasseringen:
dp.write_pandas(pd.DataFrame(columns = ["Date", "User", "Json-dataframe"]), "{path}", valuation="INTERNAL", state= "OTHER")
''')
        # Kan være andre errorer her vi ikke har forutsett, så skriv ut errorbeskjeden også
        print(e)

In [ ]:
# Her kaller vi write-funksjonen over, med eksempeldataen
microlog_write(diam, diam_edit, microlog_path)

In [ ]:
# NB FARLIG - overskriver / "tømmer" loggen, evt. opprett den første gang du 
#dp.write_pandas(pd.DataFrame(columns = ["Date", "User", "Json-dataframe"]), microlog_path, valuation="INTERNAL", state= "OTHER")

In [ ]:
# Funksjon for å se på de n-antall siste entryene i den totale loggen.
# Num er her antall log-rader du vil hente, default er 1
def microlog_print(path, num = 1):
    # Siden denne funksjonen i teorien kan kopieres inn dit man ønsker, så forsikrer vi oss bare om at vi har de importene vi trenger.
    import dapla as dp
    from IPython.core.display import display, HTML
    
    # Hent fra daplalagring, og begrens return.
    micro_sample = dp.read_pandas(path).tail(num)
    
    # Loop gjennom entryene vi henter (default 1)
    for index, row in micro_sample.iterrows():
        # Print ut brukernavn og tidspunkt
        display(HTML(f'Redigert av {row["User"]}\nTidspunkt: {row["Date"]}\n'))
        # Konverter dataframen ut av en celle som json, tilbake til en dataframe
        display(pd.read_json(row['Json-dataframe']))
        # to_json() og read_json() støtter ikke multiindexing i utgangspunktet, så resultatet kan se litt annerledes ut enn ut fra mikrolog_write()-display
    

In [ ]:
# Her kaller vi print-funksjonen over på pathen vi lagrer hoveddatframen til
microlog_print(microlog_path, 2)